In [ ]:
import pyecap
import os
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.io as sio
from scipy.signal import find_peaks
from fnirs_functions import *

In [ ]:
meta_index = 4
metaDF = pd.read_excel(r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\metadata.xlsx')
tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']

#flex_folder = metaDF.loc[meta_index, 'fNIRs Folder'] + '.mat'
flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

fd = flexNIRs(flex_folder)

ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

stim = pyecap.Stim(tank)
stimDF = stim.parameters

In [ ]:
"""Align fNIRs Artifact"""
alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
fd.plot_artifact(channel = 'D1 Ambient', fc = 1, filter_type = 'highpass', show_stim = True)

In [ ]:
"""Performs short-channel regression to generate SSR-DF"""
data_type = 'Hemo_filt' # Data type of original data to run SSR regression on
fd.ssr_regression(data_type = data_type)
fd.ssr_plot(data_type, channel = 'D1 LS HbT', show_stim=True, show_ss=True)

In [ ]:
data_type = 'SSR_filt'
channels = ['D1 LS', 'D1 LL', 'D3 LS', 'D3 LL']

for chan in channels:
    fd.gigaplot(data_type, channel = chan, pre_time=5, post_time=5,show_stim=True, fig_size = (12,6), zero_shift = True)

In [ ]:
#fd.ssrDF_f.columns[0]
fd.ssrDF_f.columns

In [ ]:
channel = 'D1 LL'
channel_pair = [ fd.ch_dct[ch][0] + ' ' + fd.ch_dct[ch][1] + ' SSR' for ch in fd.ch_pairs[channel]]

channel_pair = [ch.replace('Red', 'HbO') for ch in channel_pair]
channel_pair = [ch.replace('IR', 'HbR') for ch in channel_pair]

print(channel_pair)

In [ ]:
fd.plot_channel(data_type = 'SSR_filt', channel = 'D3 LL', pre_time =5, post_time = 5, plot_style='Full', fig_size =(4,10), zero_shift = True)

In [ ]:
"""Perform HR calculations"""
#fNIRs HR
fd.calc_hr(channel = 'SS Red', filter_cutoffs=(0.5,15),peak_height = 2000, check_plot = False)

#Filter TDT ECG without Down-sampling
tdt_ecg = filter_hr(ecg_data, cutoff = 10, downsample=False, check_plot=False)
ecgDF = calc_hr(tdt_ecg, peak_height = 250, fs = ecg_data.sample_rate)

#Filter and down sample TDT ECG data for Plotting
DS_factor = 10
tdt_ecg_DS = filter_hr(ecg_data, cutoff = 10, downsample=True, downsample_factor = DS_factor, check_plot=False)
ecgDF_DS = calc_hr(tdt_ecg_DS, peak_height = 250, fs = ecg_data.sample_rate / DS_factor)

In [ ]:
"""Plot ECG"""
fig = go.Figure()
fig.add_trace(go.Scatter(x = ecgDF['Time (s)'], y = ecgDF['smooth bpm'], name = 'TDT HR'))
#fig.add_trace(go.Scatter(x = fd.ecgDF['Time (s)'], y = fd.ecgDF['b2b bpm'],name = 'fNIRs HR'))
#fig.add_trace(go.Scatter(x = ecgDF_DS['Time (s)'], y = ecgDF_DS['b2b bpm'], name = 'downsampled'))
for param in stimDF.index:
        fig.add_vrect(x0=fd.stimDF.loc[param]['onset time (s)'],
                  x1=fd.stimDF.loc[param]['offset time (s)'],
                      line_width=0, fillcolor='red', opacity=0.15)
fig.show()

In [ ]:
fd.plot_channel_interactive(data_type = 'SSR_filt', channel = 'D1 LL Red SSR', show_stim =True, show_hr=True, hr_chan='smooth')

In [ ]:
data_type = 'Mua_filt' # Data type of original data
fd.ssr_regression(data_type = data_type)
#fd.ssr_plot(data_type, channel = 'D1 LL Red', show_stim=True, show_ss=True)

In [ ]:
"""Plot for data presentation
Plots of interest:
HbO/HbR or each channel with different stims
Priority: HbT of each channel of different stim parameters overlaid on top of each to assess differences in stim amplitude effects?
Plot showing effect of SSR regression data
Add HR as a secondary plot?
"""
fd.plot_channel(data_type = 'SSR_filt', channel = 'D1 LL', pre_time =5, post_time = 5, plot_style='Full', fig_size =(4,10), zero_shift = True)

In [ ]:
"""Stacked ECG plot"""
#metaLIST = [5,2,3,4] #3 mA
#metaLIST = [6,8,7,9] #4 mA
metaLIST = [12,11,13,10] #6 mA

freqLIST = [17, 100,200,300] #3 mA

fig,ax = plt.subplots(nrows = 4, figsize = (10,10), layout='constrained')
ax = ax.ravel()

for idx, meta_index in enumerate(metaLIST):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    #Filter TDT ECG without Down-sampling
    tdt_ecg = filter_hr(ecg_data, cutoff = 10, downsample=False, check_plot=False)
    ecgDF = calc_hr(tdt_ecg, peak_height = 250, fs = ecg_data.sample_rate)

    title = '6 mA, ' + str(freqLIST[idx]) + ' Hz'
    ax[idx].set_xlim( stimDF.loc[stimDF.index[0], 'onset time (s)'] - 60, stimDF.loc[stimDF.index[-1], 'offset time (s)'] + 60 )
    sns.lineplot(data = ecgDF, x = ecgDF['Time (s)'], y = ecgDF['smooth bpm'], ax=ax[idx], label = 'HR (bpm)').set_title(title)
    ax[idx].set_ylim(50,70)

    for param in stimDF.index:
        ax[idx].axvspan(xmin=stimDF.loc[param]['onset time (s)'],
                                            xmax=stimDF.loc[param]['offset time (s)'], color='red', alpha=0.1)

In [ ]:
fs = fd.fs
pre_time = 5
post_time = 5
zero_shift = True

ss_chan = 'SS HbT'

channels = ['D1 LS HbT SSR', 'D1 LL HbT SSR', 'D3 LS HbT SSR', 'D3 LL HbT SSR']

pre_BL_idx_width = int(pre_time * fs)
post_BL_idx_width = int(post_time * fs)

for chan in channels:

    plotDF = fd.get_data('SSR_filt')[0]
    plotDF['Time (s)'] = fd.time

    #Add SS HbT channel
    ssDF = fd.get_data('Hemo_filt')[0]
    ssDF['Time (s)'] = fd.time

    stimDF = fd.stimDF.copy()
    # Label data points based on stimulation #
    for idx, stim in enumerate(stimDF.index):
        stim_start_idx = stimDF.loc[stim, 'fNIRs onset index']
        stim_stop_idx = stimDF.loc[stim, 'fNIRs offset index']

        plot_start_idx = (stim_start_idx - pre_BL_idx_width).astype(int)
        plot_stop_idx = (stim_stop_idx + post_BL_idx_width).astype(int)

        if plot_stop_idx > len(plotDF):
            raise Exception('Plot stop index exceeds plot length. Reduce post-stimulation time.')

        plotDF.loc[plot_start_idx:plot_stop_idx, 'Stim #'] = str(idx + 1)
        ssDF.loc[plot_start_idx:plot_stop_idx, 'Stim #'] = str(idx + 1)

        # Construct time array based on sampling frequency and number of indices with time zero at stimulation start
        time_span = plot_stop_idx - plot_start_idx + 1
        time = np.arange(-pre_BL_idx_width, time_span - pre_BL_idx_width, 1) / fs

        plotDF.loc[plot_start_idx:plot_stop_idx, 'Trial Time'] = time
        ssDF.loc[plot_start_idx:plot_stop_idx, 'Trial Time'] = time

    #Construct super figure
    fig = plt.figure(layout='constrained', figsize=(9,6))

    # mosaic = [['full', 'full', 'full', 'full'],
    #           ['D1 LS avg', 'D1 LL avg', 'D3 LS avg', 'D3 LL avg'],
    #           ['D1 LS Red', 'D1 LL Red', 'D3 LS Red', 'D3 LL Red'],
    #           ['D1 LS IR', 'D1 LL IR', 'D3 LS IR', 'D3 LL IR'], ]

    mosaic = [['full','full', 'full', 'full'],
          ['avg', 'Red','avg SS', 'SS stack']]

    ax_dict = fig.subplot_mosaic(mosaic)

    #Top Plot = Whole trace with shaded stim boxed

    sns.lineplot(data = plotDF, x = plotDF['Time (s)'], y = plotDF[chan], ax = ax_dict['full'], label = chan)
    sns.lineplot(data = ssDF, x = ssDF['Time (s)'], y = ssDF[ss_chan], ax = ax_dict['full'], label = 'SS HbT')
    for param in stimDF.index:
        ax_dict['full'].axvspan(xmin=stimDF.loc[param]['fNIRs onset time (s)'],
                                xmax=stimDF.loc[param]['fNIRs offset time (s)'], color='red', alpha=0.05)

    plotDF.dropna(axis=0, subset=['Stim #'], inplace=True)
    ssDF.dropna(axis=0, subset=['Stim #'], inplace=True)

    # Avg. Response during stim Plot
    if zero_shift:
        for stim_num in plotDF['Stim #'].unique():
            zero_point = plotDF.loc[plotDF['Stim #'] == stim_num, 'Trial Time'].abs().idxmin()
            plotDF.loc[plotDF['Stim #'] == stim_num, chan] = (
                    plotDF.loc[plotDF['Stim #'] == stim_num, chan] - plotDF.loc[
                zero_point, chan].item())
            
            ss_zero_point = ssDF.loc[ssDF['Stim #'] == stim_num, 'Trial Time'].abs().idxmin()
            ssDF.loc[ssDF['Stim #'] == stim_num, ss_chan] = (
                    ssDF.loc[ssDF['Stim #'] == stim_num, ss_chan] - ssDF.loc[
                ss_zero_point, ss_chan].item())

            
    sns.lineplot(data=plotDF, x='Trial Time', y=chan, errorbar='sd', ax=ax_dict['avg'], label = 'HbT')
    sns.lineplot(data=plotDF, x='Trial Time', y=chan, hue='Stim #', ax=ax_dict['Red'], legend = False)
    
    sns.lineplot(data=ssDF, x='Trial Time', y=ss_chan, errorbar='sd', ax=ax_dict['avg SS'], label = 'SS HbT')
    sns.lineplot(data=ssDF, x='Trial Time', y=ss_chan, hue = 'Stim #', ax=ax_dict['SS stack'], legend = False)

    ax_dict['avg'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['Red'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['avg SS'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['SS stack'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)